# Correlation Between Public Transport and Traffic of Congestion Charge Zone in Greater London Area

## Public Transport Journeys Type
source: https://data.london.gov.uk/dataset/public-transport-journeys-type-transport

## Vehicles Entering C-Charge Zone in Greater London Area
source :https://data.london.gov.uk/dataset/vehicles-entering-c-charge-zone-month

In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [186]:
publicTransport = pd.read_csv("tfl-journeys-type.csv")
publicTransport.head()



,Period and Financial year,Reporting Period,Days in period,Period beginning,Period ending,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m)
0,01_10/11,1,31,01-Apr-10,01-May-10,189.1,90.5,6.3,2.3,NaN,NaN,NaN
1,02_10/11,2,28,02-May-10,29-May-10,181.6,84.5,5.8,2.2,NaN,NaN,NaN
2,03_10/11,3,28,30-May-10,26-Jun-10,175.9,84.3,5.8,2.1,NaN,NaN,NaN
3,04_10/11,4,28,27-Jun-10,24-Jul-10,183.4,86.5,6.1,2.1,NaN,NaN,NaN
4,05_10/11,5,28,25-Jul-10,21-Aug-10,160.4,82.9,5.8,2.0,NaN,NaN,NaN


In [187]:
publicTransport.tail()

,Period and Financial year,Reporting Period,Days in period,Period beginning,Period ending,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m)
170,02_23/24,2,28,30-Apr-23,27-May-23,146.0,88.2,7.7,1.6,14.1,0.1,15.1
171,03_23/24,3,28,28-May-23,24-Jun-23,145.8,90.2,7.9,1.7,14.6,0.1,16.0
172,04_23/24,4,28,25-Jun-23,22-Jul-23,147.1,94.1,7.9,1.6,14.3,0.1,16.7
173,05_23/24,5,28,23-Jul-23,19-Aug-23,129.7,86.6,7.2,1.4,12.4,0.2,15.4
174,06_23/24,6,28,20-Aug-23,16-Sep-23,139.8,83.7,7.3,1.7,13.8,0.2,15.6


In [188]:
traffic = pd.read_csv('tfl-vehicles-c-charge-zone.csv')

traffic.head()

,Month,CC Camera Captures during Charging Hours,CC Confirmed Vehicles observed during Charging Hours,Number of Charging Day in Month,Notes
0,Jul-10,5615063,NaN,22,NaN
1,Aug-10,5174428,NaN,21,30 Aug Non Charging Day
2,Sep-10,6198427,NaN,22,NaN
3,Oct-10,6001932,NaN,21,NaN
4,Nov-10,6430184,NaN,22,NaN


In [189]:
traffic.tail()

,Month,CC Camera Captures during Charging Hours,CC Confirmed Vehicles observed during Charging Hours,Number of Charging Day in Month,Notes
154,May-23,2579290,2308362.0,31,NaN
155,Jun-23,2763319,2443181.0,30,NaN
156,Jul-23,2710014,2316631.0,31,NaN
157,Aug-23,2563401,2277610.0,31,NaN
158,Sep-23,2721549,2439795.0,30,NaN


# Data Cleaning

Drop rows before July 2010 of publicTransport dataset

In [190]:
publicTransport_clean = publicTransport.drop([0,1,2])
publicTransport_clean = publicTransport_clean.reset_index().drop(columns=['index'])
publicTransport_clean.head()


,Period and Financial year,Reporting Period,Days in period,Period beginning,Period ending,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m)
0,04_10/11,4,28,27-Jun-10,24-Jul-10,183.4,86.5,6.1,2.1,NaN,NaN,NaN
1,05_10/11,5,28,25-Jul-10,21-Aug-10,160.4,82.9,5.8,2.0,NaN,NaN,NaN
2,06_10/11,6,28,22-Aug-10,18-Sep-10,175.8,80.9,5.5,2.0,NaN,NaN,NaN
3,07_10/11,7,28,19-Sep-10,16-Oct-10,189.8,88.7,6.3,2.3,NaN,NaN,NaN
4,08_10/11,8,28,17-Oct-10,13-Nov-10,179.9,90.3,6.7,2.2,5.6,NaN,NaN


In [191]:
publicTransport_clean = publicTransport_clean.fillna(0)
publicTransport_clean.head()

,Period and Financial year,Reporting Period,Days in period,Period beginning,Period ending,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m)
0,04_10/11,4,28,27-Jun-10,24-Jul-10,183.4,86.5,6.1,2.1,0.0,0.0,0.0
1,05_10/11,5,28,25-Jul-10,21-Aug-10,160.4,82.9,5.8,2.0,0.0,0.0,0.0
2,06_10/11,6,28,22-Aug-10,18-Sep-10,175.8,80.9,5.5,2.0,0.0,0.0,0.0
3,07_10/11,7,28,19-Sep-10,16-Oct-10,189.8,88.7,6.3,2.3,0.0,0.0,0.0
4,08_10/11,8,28,17-Oct-10,13-Nov-10,179.9,90.3,6.7,2.2,5.6,0.0,0.0


In [192]:
tflJourneys = pd.DataFrame(columns=['Month', 'Bus journeys (m)','Underground journeys (m)','DLR Journeys (m)','Tram Journeys (m)','Overground Journeys (m)','London Cable Car Journeys (m)','TfL Rail Journeys (m)','Total journeys (m)'])
tflJourneys.head()

,Month,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m),Total journeys (m)


In [193]:

for index, row in publicTransport_clean.iterrows():
    if(row['Period beginning'][3:6] != row['Period ending'][3:6]) :
        if(row['Period ending'][3:6] == publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['Period beginning'][3:6]):
            bus = (row['Bus journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['Bus journeys (m)'])/2
            underground = (row['Underground journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['Underground journeys (m)'])/2
            DLR = (row['DLR Journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['DLR Journeys (m)'])/2
            tram = (row['Tram Journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['Tram Journeys (m)'])/2
            overground = (row['Overground Journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['Overground Journeys (m)'])/2
            cable = (row['London Cable Car Journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['London Cable Car Journeys (m)'])/2
            rail = (row['TfL Rail Journeys (m)'] + publicTransport_clean.iloc[min(index+1, len(publicTransport_clean)-1)]['TfL Rail Journeys (m)'])/2
            total = bus + underground + DLR + tram + overground + cable + rail


            tflJourneys.loc[index] = ({'Month' : row['Period ending'][3:10] , 'Bus journeys (m)': bus, 'Underground journeys (m)' : underground, 'DLR Journeys (m)' : DLR, 'Tram Journeys (m)': tram, 'Overground Journeys (m)': overground, 'London Cable Car Journeys (m)': cable, 'TfL Rail Journeys (m)': rail, 'Total journeys (m)' : total})


In [194]:
tflJourneys

,Month,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m),Total journeys (m)
0,Jul-10,171.90,84.70,5.95,2.05,0.00,0.00,0.00,264.60
1,Aug-10,168.10,81.90,5.65,2.00,0.00,0.00,0.00,257.65
2,Sep-10,182.80,84.80,5.90,2.15,0.00,0.00,0.00,275.65
3,Oct-10,184.85,89.50,6.50,2.25,2.80,0.00,0.00,285.90
4,Nov-10,179.35,90.45,6.55,2.25,5.50,0.00,0.00,284.10
...,...,...,...,...,...,...,...,...,...
164,Mar-23,141.90,87.35,7.70,1.65,13.30,0.10,14.00,266.00
167,May-23,145.90,89.20,7.80,1.65,14.35,0.10,15.55,274.55
168,Jun-23,146.45,92.15,7.90,1.65,14.45,0.10,16.35,279.05
169,Jul-23,138.40,90.35,7.55,1.50,13.35,0.15,16.05,267.35


In [195]:
idx = tflJourneys.loc[tflJourneys['Month']=='Oct-16'].index
tflJourneys = tflJourneys.loc[idx[0]:]
tflJourneys.reset_index(drop=True, inplace=True)
tflJourneys


,Month,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m),Total journeys (m)
0,Oct-16,181.75,111.50,9.85,2.35,15.30,0.10,3.75,324.60
1,Nov-16,179.65,114.65,10.10,2.15,15.55,0.10,3.90,326.10
2,Dec-16,164.25,102.85,8.85,2.10,13.85,0.10,3.65,295.65
3,Jan-17,160.10,95.45,8.60,2.25,13.40,0.10,3.55,283.45
4,Feb-17,173.65,105.10,9.65,2.30,14.70,0.10,3.70,309.20
...,...,...,...,...,...,...,...,...,...
69,Mar-23,141.90,87.35,7.70,1.65,13.30,0.10,14.00,266.00
70,May-23,145.90,89.20,7.80,1.65,14.35,0.10,15.55,274.55
71,Jun-23,146.45,92.15,7.90,1.65,14.45,0.10,16.35,279.05
72,Jul-23,138.40,90.35,7.55,1.50,13.35,0.15,16.05,267.35


In [196]:

traffic.dropna(subset=['CC Confirmed Vehicles observed during Charging Hours'], inplace=True)

numVehicles = traffic.loc[:,['Month','CC Confirmed Vehicles observed during Charging Hours']]
numVehicles.reset_index(drop=True, inplace=True)
numVehicles 


,Month,CC Confirmed Vehicles observed during Charging Hours
0,Oct-16,2283444.0
1,Nov-16,2396023.0
2,Dec-16,1794886.0
3,Jan-17,2119813.0
4,Feb-17,2049863.0
...,...,...
79,May-23,2308362.0
80,Jun-23,2443181.0
81,Jul-23,2316631.0
82,Aug-23,2277610.0


# Data Integration

In [197]:
df = tflJourneys.join(numVehicles.set_index('Month'), on='Month')
df = df.set_index('Month')
df

,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),London Cable Car Journeys (m),TfL Rail Journeys (m),Total journeys (m),CC Confirmed Vehicles observed during Charging Hours
Month,,,,,,,,,
Oct-16,181.75,111.50,9.85,2.35,15.30,0.10,3.75,324.60,2283444.0
Nov-16,179.65,114.65,10.10,2.15,15.55,0.10,3.90,326.10,2396023.0
Dec-16,164.25,102.85,8.85,2.10,13.85,0.10,3.65,295.65,1794886.0
Jan-17,160.10,95.45,8.60,2.25,13.40,0.10,3.55,283.45,2119813.0
Feb-17,173.65,105.10,9.65,2.30,14.70,0.10,3.70,309.20,2049863.0
...,...,...,...,...,...,...,...,...,...
Mar-23,141.90,87.35,7.70,1.65,13.30,0.10,14.00,266.00,2377372.0
May-23,145.90,89.20,7.80,1.65,14.35,0.10,15.55,274.55,2308362.0
Jun-23,146.45,92.15,7.90,1.65,14.45,0.10,16.35,279.05,2443181.0


# Data Selection

In [198]:
X = df.iloc[:,:-1]
y = df['CC Confirmed Vehicles observed during Charging Hours']

X.shape, y.shape

((74, 8), (74,))

In [199]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((59, 8), (15, 8), (59,), (15,))

## Feature Scaling

Using StandardScaler to scale features to ensure _equal contribution of features_ and _improved convergence_.

In [200]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Feature Selection

1.  Recursive Feature Elimination (RFE)

In [201]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

# Create a random forest regression model
model = RandomForestRegressor(random_state=42)

rfe = RFE(model, n_features_to_select=5)  # Set the desired number of features
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

# Get the mask of selected features
selected_features_mask = rfe.support_

# Get the feature names of the selected features
selected_features_rfe = [feature for feature, selected in zip(X.columns, selected_features_mask) if selected]
selected_features_rfe

['Bus journeys (m)',
 'Underground journeys (m)',
 'DLR Journeys (m)',
 'Tram Journeys (m)',
 'TfL Rail Journeys (m)']

2. Least Absolute Shrinkage and Selection Operator (LASSO)

In [202]:
from sklearn.linear_model import Lasso


lasso_model = Lasso(alpha=0.1, max_iter=2000) 

lasso_model.fit(X_train, y_train)

X_train_lasso = X_train[:, lasso_model.coef_ != 0]
X_test_lasso = X_test[:, lasso_model.coef_ != 0]

selected_features_lasso = [feature for feature, coef in zip(X.columns, lasso_model.coef_) if coef != 0]
selected_features_lasso

/home/adhl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.793e+11, tolerance: 7.130e+08
  model = cd_fast.enet_coordinate_descent(


['Bus journeys (m)',
 'Underground journeys (m)',
 'DLR Journeys (m)',
 'Tram Journeys (m)',
 'Overground Journeys (m)',
 'London Cable Car Journeys (m)',
 'TfL Rail Journeys (m)',
 'Total journeys (m)']

Based on both methods, the Top 5 contributing features in the dataset are:
 - Bus journeys (m)
 - Underground journeys (m)
 - DLR Journeys (m)
 - Tram Journeys (m)
 - Overground Journeys (m)

In [203]:
X_train = X_train_lasso
X_test = X_test_lasso

# Data Mining

1. Support Vector Machine Regressor (SVR)

In [204]:
from sklearn.svm import SVR
svr = SVR(kernel = 'linear')
svr.fit(X_train, y_train)
svr.score(X_test, y_test)

-0.041439100028042075

Using Decision Tree for Regression Learning

In [205]:
from sklearn.tree import DecisionTreeRegressor
regressionTree = DecisionTreeRegressor(random_state=42)
regressionTree.fit(X_train, y_train)
regressionTree.score(X_test, y_test)

0.3000482461642383

## Ensemble Learning

### Bagging with Random Forest

In [206]:
from  sklearn.ensemble import RandomForestRegressor
randomForest = RandomForestRegressor(random_state=42)
randomForest.fit(X_train, y_train)
randomForest.score(X_test, y_test)

0.45229873175538193

### Boosting with AdaBoost

In [207]:
from  sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor(random_state=42)
adaBoost.fit(X_train, y_train)
adaBoost.score(X_test, y_test)

0.40404592497558656

### Boosting with GradientBoostingRegressor

In [218]:
from  sklearn.ensemble import GradientBoostingRegressor
gradientBoost = GradientBoostingRegressor(random_state=42)
gradientBoost.fit(X_train, y_train)
gradientBoost.score(X_test, y_test)

0.5228608012558678

Hyperparameter Tuning

In [229]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=42)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_gb_model = grid_search.best_estimator_
best_gb_model.fit(X_train, y_train)
best_gb_model.score(X_test, y_test)


0.42975223202370516

In [228]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(gradientBoost, X_train, y_train, cv=5)

scores


array([0.62438253, 0.29691511, 0.54851546, 0.53417241, 0.57131792])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GradientBoostingRegressor
gb_model = GradientBoostingRegressor()

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Get the best-performing model
best_gb_model = grid_search.best_estimator_

# Train the best model on the entire training set
best_gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_gb_model.predict(X_test)

# Evaluate the best model
mse = mean_squared_error(y_test, y_pred)
print("Best Model Mean Squared Error:", mse)


# Pattern Evaluation

Based on the ML models, it can be concluded that 

# Knowledge Presentation